<a href="https://colab.research.google.com/github/claude9493/DSAA5002/blob/main/notebook/BetaRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/claude9493/dsaa5002.git
!mv ./dsaa5002/dataset/ ./
!mv ./dsaa5002/utils/ ./

Cloning into 'dsaa5002'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [2]:
%%time

from utils.utils import create_dataset, Trainer
# from layer.layer import Embedding, FeaturesEmbedding, EmbeddingsInteraction, MultiLayerPerceptron

import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Training on [{}].'.format(device))

Training on [cpu].
CPU times: user 1.02 s, sys: 1.06 s, total: 2.08 s
Wall time: 27.7 s


In [3]:
%%time
SAMPLE_NUM = 10000
task = 'regression'  # 'classification'
dataset = create_dataset('movielens', sample_num=SAMPLE_NUM, task=task, device=device)
field_dims, (train_X, train_y), (valid_X, valid_y), (test_X, test_y) = dataset.train_valid_test_split()

CPU times: user 17.3 ms, sys: 2.41 ms, total: 19.7 ms
Wall time: 79.4 ms


In [ ]:
class Beta_Loss:
    def __init__(self, model, reg_biase=0.005, reg_lambda=0.005):
        self.reg_biase = reg_biase
        self.reg_lambda = reg_lambda
        self.model = model

    def __call__(self, pred, target):
        loss = (1-self.reg_biase - self.reg_lambda) * nn.MSELoss()(pred, target)
        loss += self.reg_biase * self.model.loss[0]
        loss += self.reg_lambda * self.model.loss[1]
        return loss


class BetaRecommendation(nn.Module):

    def __init__(self, field_dims, embed_dim=4):
        super(BetaRecommendation, self).__init__()
        n_users, n_movies = field_dims[0], field_dims[1]

        self.Bu = nn.Parameter(torch.randn(n_users), requires_grad=True)
        self.Bm = nn.Parameter(torch.randn(n_movies), requires_grad=True)

        self.u = nn.Embedding(n_users, embed_dim)
        self.m = nn.Embedding(n_movies, embed_dim)
        
        self.u.weight.data.uniform_(0, 0.05)
        self.m.weight.data.uniform_(0, 0.05)
        self.loss = [0,0]

        
    def forward(self, x, global_mean=0):
        users, movies = x[:,0], x[:,1]
        u, m = self.u(users), self.m(movies)
        Bu, Bm = self.Bu[users], self.Bm[movies]
        alpha_u, beta_u = torch.chunk(u, 2, dim=-1)
        alpha_m, beta_m = torch.chunk(m, 2, dim=-1)

        u_dist = torch.distributions.beta.Beta(alpha_u, beta_u)
        m_dist = touch.distributions.beta.Beta(alpha_m, beta_m)

        distance = self.distance(u_dist, m_dist)
    
        output = global_mean + Bu + Bm - distance
        
        self.loss[0] = torch.norm(Bu) + torch.norm(Bm)
        # self.loss[1] = torch.norm(u) + torch.norm(m)
        return output
      
    def distance(self, u_dist, m_dist):
      return torch.norm(math.pi/2.0 * 
                        torch.atan(
                            torch.distributions.kl.kl_divergence(
                                u_dist, m_dist)), 
                        p=1, dim=-1)

In [ ]:
%%time

EMBEDDING_DIM = 8
LEARNING_RATE = 1e-4
REGULARIZATION = 1e-6
BATCH_SIZE = 1024
EPOCH = 5000
TRIAL = 1000

br = BetaRecommendation(field_dims, EMBEDDING_DIM).to(device)

optimizer = optim.Adam(br.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)
# criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()
criterion = EE_Loss(model=br)

trainer = Trainer(br, optimizer, criterion, BATCH_SIZE, task=task)
trainer.train(train_X, train_y, epoch=EPOCH, trials=TRIAL, valid_X=valid_X, valid_y=valid_y)
test_loss, test_metric = trainer.test(test_X, test_y)
print('test_loss:  {:.5f} | test_metric:  {:.5f}'.format(test_loss, test_metric))